# Imports

In [1]:
import numpy as np
import csv
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.interpolate import InterpolatedUnivariateSpline
from plotly.subplots import make_subplots
import plotly.express as px
from plotly import graph_objs as go

import os
from joblib import Parallel, delayed
import time
from time import sleep
import datetime
from tqdm.notebook import trange

from IPython.display import clear_output
from collections import defaultdict
import plotly.io as pio
import multiprocessing
import copy
import random

# Figure Template

In [2]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'title': 'Time (\u03BCs)',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'title': 'Coherence',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 16,
            },
    'colorway': ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
}

# Import Experiemnt data

In [3]:
# Total 12 qubits from IBM-Algiers
expt_data = np.load('/Users/bhavesh/bhavesh/codes/ML_DD-main/data/T1_T2_data.npz',allow_pickle=True)

In [4]:
expt_data.files

['pop_t1',
 'pop_t2_hahn',
 'pop_t2_X32',
 'pop_t2_XY84',
 'xval',
 'counts_t1',
 'counts_t2_hahn',
 'counts_t2_X32',
 'counts_t2_XY84']

In [5]:
# 5 times repetated data for qubit number index as 8 and 11 on exp_data file 
expt_rep_data = np.load('/Users/bhavesh/bhavesh/codes/ML_DD-main/data/T1_T2_time_dep_data.npz',allow_pickle=True)

In [6]:
expt_rep_data.files

['pop_t1', 'pop_t2_X32', 'xval', 'counts_t1', 'counts_t2_X32']

In [7]:
(expt_data['xval']-expt_rep_data['xval']) # 20th Feb - 2nd March, note that difference is not zero

array([ 4.97777778e-08, -3.20000000e-08,  1.10222222e-07,  2.48888889e-08,
       -6.75555556e-08,  6.75555556e-08, -3.20000000e-08,  9.24444444e-08,
       -1.42222222e-08,  1.03111111e-07, -1.06666667e-08,  9.60000000e-08,
       -2.84444444e-08,  7.11111111e-08, -6.40000000e-08,  2.48888889e-08,
        1.06666667e-07, -4.62222222e-08,  2.48888889e-08,  8.88888889e-08,
       -8.53333333e-08, -3.20000000e-08,  1.06666667e-08,  4.62222222e-08,
        7.11111111e-08,  9.24444444e-08,  1.03111111e-07,  1.06666667e-07,
        9.95555556e-08,  8.17777778e-08,  5.68888889e-08,  2.13333333e-08,
       -2.48888889e-08, -8.17777778e-08,  7.46666667e-08, -7.11111111e-09,
       -9.95555556e-08,  2.13333333e-08, -9.95555556e-08, -7.11111111e-09,
        7.46666667e-08, -8.88888889e-08, -4.26666667e-08, -1.06666667e-08,
        3.55555556e-09,  0.00000000e+00, -2.13333333e-08, -6.04444444e-08,
        1.03111111e-07,  2.48888889e-08, -8.17777778e-08,  2.13333333e-08,
        9.95555556e-08, -

# Normalization of T2 signals

In [8]:
# Normalise experimental decoherence data from 1 to 0
def normalise(data):
  return (data-0.5)/0.5

# Normalisation of T2_X32 signals
exp_X32_data = normalise(expt_data['pop_t2_X32'])
norm_X32_data = np.zeros(expt_data['pop_t2_X32'].shape)
for i in range(expt_data['pop_t2_X32'].shape[0]):
  norm_X32_data[i,:] = exp_X32_data[i,:]/exp_X32_data[i,:3].mean()
  
# Normalisation of T2_X32_rep signals
exp_X32_data_rep = normalise(expt_rep_data['pop_t2_X32'])
norm_X32_data_rep = np.zeros(expt_rep_data['pop_t2_X32'].shape)
for i in range(expt_rep_data['pop_t2_X32'].shape[0]):
  norm_X32_data_rep[i,:] = exp_X32_data_rep[i,:]/exp_X32_data_rep[i,:3].mean()

# Fitting for X32 over sqrt T1 signals

Extraction of T1 and T2,p and amplitude fitting parameters by stretched fitting exponentials functions

In [9]:
# exponential function fit
def simpleExp(T0,T1):
    C = np.exp(-(T0/T1))
    return C
  
# Extract T1 by fitting the data with simple-exponential function
def fit_simpleExp(signal,T0):
    params = curve_fit(simpleExp, T0, signal, bounds=([10e-6],[1000e-6]))
    T1 = params[0]
    T1err = np.sqrt(np.diag(params[1]))
    return T1, T1err

# Stretched-exponential function fit
def stretchExp(T0,T2,p,A):
    C = A*np.exp(-((T0/T2)**p))
    return C

# Extract T2, p, and amplitude by fitting the data with stretched-exponential function
def fit_stretchExp(C,T0):
    params = curve_fit(stretchExp, T0, C, bounds=([100e-6,1,0.97],[400e-6,2,1.03]))
    T2, p, A = params[0]
    T2err, perr, Aerr = np.sqrt(np.diag(params[1]))
    return T2, p, A, T2err, perr, Aerr

In [10]:
# Fitting expt_X32_data to extract T2 and p values
param_X32 = np.zeros((3,expt_data['pop_t1'].shape[0])) # norm_T2_X32 over sqrt-T1 fit parameters
fit_curves_X32 = np.zeros((expt_data['pop_t1'].shape[1],expt_data['pop_t1'].shape[0]))
for i in range(expt_data['pop_t1'].shape[0]):
  param_X32[0,i],param_X32[1,i],param_X32[2,i],_,_,_ = fit_stretchExp(np.squeeze(norm_X32_data[i,:]/np.sqrt(expt_data['pop_t1'])[i,:]),expt_data['xval'])
  fit_curves_X32[:,i] = stretchExp(expt_data['xval'],param_X32[0,i],param_X32[1,i],param_X32[2,i])
  
# Fitting expt_X32_data_rep to extract T2 and p values
param_X32_rep = np.zeros((3,expt_rep_data['pop_t1'].shape[0])) # norm_T2_X32 over sqrt-T1 fit parameters
fit_curves_X32_rep = np.zeros((expt_rep_data['pop_t1'].shape[1],expt_rep_data['pop_t1'].shape[0]))
for i in range(expt_rep_data['pop_t1'].shape[0]):
  param_X32_rep[0,i],param_X32_rep[1,i],param_X32_rep[2,i],_,_,_ = fit_stretchExp(np.squeeze(norm_X32_data_rep[i,:]/np.sqrt(expt_rep_data['pop_t1'])[i,:]),expt_rep_data['xval'])
  fit_curves_X32_rep[:,i] = stretchExp(expt_rep_data['xval'],param_X32_rep[0,i],param_X32_rep[1,i],param_X32_rep[2,i])

In [11]:
param_X32[0,:]*1e6

array([100.        , 330.00868986, 400.        , 381.76505327,
       287.71482578, 400.        , 400.        , 174.35829477,
       304.3132239 , 223.88604908, 305.77860076, 160.24646748])

In [12]:
param_X32[1,:]

array([1.        , 1.0940438 , 1.66329449, 1.        , 1.51168977,
       1.64846917, 1.90189399, 1.        , 1.        , 1.        ,
       1.01366848, 1.34311527])

In [13]:
# Concatenation of expt_data for qubits index as [1,3,4,8,10,11] and expt_rep_data 
c_expt = np.concatenate((np.array([norm_X32_data[i,:]/np.sqrt(expt_data['pop_t1'])[i,:] for i in [1,3,4,8,10,11]]),norm_X32_data_rep/np.sqrt(expt_rep_data['pop_t1'])),axis=0)

t_expt = np.concatenate((np.array([expt_data['xval'] for i in [1,3,4,8,10,11]]),np.array([expt_rep_data['xval'] for j in range(10)])),axis=0)

In [15]:
print(t_expt.shape)
print(c_expt.shape)

(16, 150)
(16, 150)


In [16]:
fig = go.Figure()
for i in range(c_expt.shape[0]):
    fig.add_scatter(x=expt_data['xval']*1e6,y=c_expt[i,:],name=f'Qubit {i}',mode="lines+markers",opacity=0.5)

fig.update_layout(template=fig_template, width = 700,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,700]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
# saving final expt_data with all treatment taken care
np.savez_compressed('/Users/bhavesh/bhavesh/codes/ML_DD-main/data'+F"/expt_data",c_expt=c_expt,xval=expt_data['xval'])

# Plot to see T2_Hahn among qubits

In [17]:
fig4 = go.Figure()
for i in range(expt_data['pop_t1'].shape[0]):
  fig4.add_scatter(x=expt_data['xval']*1e6,y=norm_X32_data[i,:]/np.sqrt(expt_data['pop_t1'])[i,:],name=f'Qubit {i}',
                  mode="lines+markers",opacity=0.5)
  fig4.add_scatter(x=expt_data['xval']*1e6,y=fit_curves_X32[:,i],name=f'{int(param_X32[0,i]*1e6)},'+f'{round(param_X32[1,i],2)}',mode="lines",line=dict(width=3,color='black'),opacity=1)

fig4.update_layout(template=fig_template, width = 700,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,700]),
                  yaxis = dict(title='Coherence'),
                 )

# Generate the filter function

In [18]:
# Create CPMG-like pulse timing array
def cpmgFilter(n, Tmax):
    tpi = np.empty([n])
    for i in range(n):
        tpi[i]= Tmax*(((i+1)-0.5)/n)
    return tpi

# Generate filter function for a given pulse sequence
def getFilter(n,w0,piLength,Tmax):
    tpi = cpmgFilter(n,Tmax)
    f = 0
    for i in range(n):
        f = ((-1)**(i+1))*(np.exp(1j*w0*tpi[i]))*np.cos((w0*piLength)/2) + f

    fFunc = (1/2)*((np.abs(1+((-1)**(n+1))*np.exp(1j*w0*Tmax)+2*f))**2)/(w0**2)
    return fFunc

# Generate the noise spectrum

In [ ]:
def transmon_noise(T2,w):
  alpha0 = 0
  alpha1 = 1
  alpha2 = 2
  # bound_cutoff = 1800
  bound_cutoff = 5500
  bound_threshold = 10000

  A0 = np.reshape(((2**alpha0)*((np.pi/T2)**(alpha0+1))),[T2.size,1])
  A1 = np.reshape(((2**alpha1)*((np.pi/T2)**(alpha1+1))),[T2.size,1])
  A2 = np.reshape(((2**alpha2)*((np.pi/T2)**(alpha2+1))),[T2.size,1])
  w = np.reshape(w,[1,w.size])

  width = 2000
  # width = 500
  s_smooth = np.zeros((T2.size,w.size-width+1))
  for i in trange(T2.size):
    wc_check = np.random.random_sample()
    if wc_check<0.25:
      # cut = -np.sort(np.array(random.sample(range(bound_cutoff,w.size),2)))
      cut1 = -np.array(random.sample(range(bound_cutoff,bound_threshold),1))
      cut2 = -np.array(random.sample(range(-cut1[0]+1,w.size),1))
      cut = np.squeeze(np.array([cut1,cut2]))
      s0 = np.squeeze(A0[i,:]/w[:,cut[0]:]**alpha0)
      s1 = np.squeeze(A1[i,:]/w[:,cut[1]:cut[0]]**alpha1)
      s1 = np.reshape(s1,(s1.size,))
      s1 = (s0[0]/s1[-1])*s1
      s2 = np.squeeze(A2[i,:]/w[:,:cut[1]]**alpha2)
      s2 = np.reshape(s2,(s2.size,))
      s2 = (s1[0]/s2[-1])*s2
      s =  0.65*np.concatenate((s2,s1,s0),axis=0)
    else:
      # cut = -np.array(random.sample(range(bound_cutoff,w.size),1))
      cut = -np.array(random.sample(range(bound_cutoff,bound_threshold),1))
      s0 = np.squeeze(A0[i,:]/w[:,cut[0]:]**alpha0)
      s1 = np.squeeze(A1[i,:]/w[:,:cut[0]]**alpha1)
      s1 = np.reshape(s1,(s1.size,))
      s1 = (s0[0]/s1[-1])*s1
      s =  0.65*np.concatenate((s1,s0),axis=0)

    s_smooth[i,:] = moving_average(s,width)
  # np.squeeze(w)[w.size-width-1:]
  return s_smooth, np.squeeze(w)[:w.size-width+1]

# Moving average
def moving_average(x, width):
    return np.convolve(x, np.ones(width), 'valid') / width

# Evaluate the Coherence curves

In [ ]:
# Generate decoherence curve corresponding to a noise spectrum (input shape = variable1.size x w.size)
def getCoherence(S,w0,T0,n,piLength):
    steps = T0.size
    C_invert = np.empty([S.shape[0],steps,])
    for i in trange(steps):
        integ = getFilter(n,np.squeeze(w0),piLength,T0[i])*S/np.pi
        integ_ans = np.trapz(y=integ,x=np.squeeze(w0))
        C_invert[:,i] = np.exp(integ_ans)
    return C_invert

# Determination of the frequency and the time array

In [ ]:
# Generate time and omega vectors
t = np.geomspace(1.8e-6,750.05e-6,451)
w_check = 32*np.pi/t
w_check.min()*1e-6, w_check.max()*1e-6

In [ ]:
w = np.flipud(np.logspace(2.0,10.0,16001))
w.min()*1e-6, w.max()*1e-6

In [ ]:
32*np.pi/w.min(), 32*np.pi/w.max()

In [ ]:
32*np.pi/(720e-6), 32*np.pi/(2e-6)

# Now data generation

In [ ]:
T2 = np.linspace(100e-6,370e-6,15001) # 100-370 microseconds with uniform spacing of 15001 
s_in, w_in = transmon_noise(T2,w)

In [ ]:
s_in.shape

In [ ]:
w_in.min()

In [ ]:
%%time
# Evaluate decoherence curves
piLength = 48e-9   # Duration of the pi-pulse
n_pulse = 32         # Number of pulses
c = getCoherence(s_in,w_in,t,n_pulse,piLength)

In [ ]:
# Fit multiple coherence curves to obtain values of T2 and p (stretching factor)
def get_fitPar(c_check,T_train):
  T2_check = []
  p_check = []
  for i in trange(c_check.shape[0]):
    T2_check0, p_check0, _, _, _, _ = fit_stretchExp(c_check[i,:],T_train)
    T2_check.append(T2_check0)
    p_check.append(p_check0)
  return np.round((np.array(T2_check)*1e6),1),np.round((np.array(p_check)),2)

In [ ]:
# to get fitting parameters
fit_par = get_fitPar(c,t)

T<sub>2</sub> Distribution

In [ ]:
hist0,x0 =np.histogram(fit_par[0],bins=51,range=[50,700],density=False)
hist1,x1 =np.histogram(fit_par[1],bins=51,range=[0.97,3],density=False)

fig = go.Figure()
fig.add_trace(go.Bar(x=x0,y=hist0,opacity=0.75,marker_color='royalblue'))
fig.update_layout(bargap=0)
fig.update_layout(template=fig_template)
fig.layout.xaxis.title = 'Fitted T<sub>2</sub> (\N{greek small letter mu}s)'
fig.layout.yaxis.title = 'Number of datasets'
fig.layout.title = 'T<sub>2</sub> Distribution'
fig

p distribution

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=x1,y=hist1,opacity=0.75,marker_color='royalblue'))
fig.update_layout(bargap=0)
fig.update_layout(template=fig_template)
fig.layout.xaxis.title = 'Stretch factor'
fig.layout.yaxis.title = 'Number of datasets'
fig.layout.title = 'p Distribution'
fig

In [ ]:
c.shape

# Prepare the data for training by adding random noise

In [ ]:
# For data interpolation
def interpData(x,y,xNew):
    f_interp = interp1d(x,y)
    yNew = f_interp(xNew)
    return yNew

# For preparing training data: Add random noise
# Run this cell multiple times to generate sets with different random noise but same underlying curves
def prepare_trainData(c_in,T_in,T_train,noiseMax=0.03):
  c_train = interpData(T_in,c_in,T_train)
  for i in trange(c_in.shape[0]):
    c_train[i,:] = c_train[i,:] + np.random.normal(0,noiseMax,size=c_train.shape[1])
  return c_train

In [ ]:
# For processing the experimental data as training data: interpolation, and random cut-off
def prepare_expData(c_in,T_in,T_train,cutOff=0.03):
  if np.max(T_train) < np.max(T_in):
    c_predict = interpData(T_in,c_in,T_train)
    for i in trange(c_in.shape[0]):
      cut = np.squeeze(np.argwhere(c_predict[i,:]<=cutOff))
      if cut.size > 1:
        c_predict[i,cut[0]-1:] = 0
      elif cut.size == 1:
        c_predict[i,cut-1:] = 0
  else:
    T_train1 = np.squeeze(T_train[np.argwhere(T_train<=np.max(T_in))])
    zero_size = T_train.size - T_train1.size
    c_predict = interpData(T_in,c_in,T_train1)
    for i in trange(c_in.shape[0]):
      cut = np.squeeze(np.argwhere(c_predict[i,:]<=cutOff))
      if cut.size > 1:
        c_predict[i,cut[0]-1:] = 0
      elif cut.size == 1:
        c_predict[i,cut-1:] = 0
    c_predict = np.concatenate((c_predict,np.zeros((c_in.shape[0],zero_size))),axis=1)
  return c_predict

In [ ]:
# Actual time vector used in the experiment (Should be smaller than t)
# T_train = np.geomspace(2e-6,720e-6,150)
T_train = expt_data['xval']

# Actual frequency vector used for training (Should be smaller than w)
w_train = np.flipud(np.geomspace(140e3,55e6,501))

In [ ]:
# Intepolate
s_train = interpData(w_in,s_in,w_train)

In [ ]:
# Interpolate coherence data if the experimental and the simulated time vectors are different
c_exp = np.zeros((norm_X32_data.shape[0],T_train.size))
for i in range(norm_X32_data.shape[0]):
  c_exp[i,:] = interpData(np.round(expt_data['xval'],10),norm_X32_data[0,:],np.round(T_train,10))

In [ ]:
# c_train = prepare_trainData(c_in,T_in,T_train)
c_train = prepare_trainData(c,t,T_train)

In [ ]:
c_train = np.zeros((5*c.shape[0],T_train.size))
s_train = np.tile(interpData(w_in,s_in,w_train),(5,1))

for i in range(5):
  print(i*c.shape[0])
  c_train[i*c.shape[0]:(i+1)*c.shape[0],:] = prepare_trainData(c,t,T_train)

Save training data as npz file

In [ ]:
np.savez_compressed(trainData_path+F"/Date_T2_noisyData",c_in=c,T_in=t,s_in=s_in,w_in=w_in,T_train=T_train,w_train=w_train)
# np.savez_compressed(trainData_path+F"/Date_T2_smoothData",c_in=c,T_in=t,s_in=s_in,w_in=w_in)

# Data Visualization

In [ ]:
c.shape

In [ ]:
t.shape

In [ ]:
c_train.shape

In [ ]:
T_train.shape

In [ ]:
color_str = ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
n_plot = 10
rand_set = np.random.randint(0,c.shape[0],(n_plot,))
exp_list = [1,3,4,8,10,11]
fig = go.Figure()
for j in exp_list:
  fig.add_scatter(x=expt_data['xval']*1e6,y=norm_X32_data[j,:]/np.sqrt(expt_data['pop_t1'])[j,:],line=dict(width=2,color='blue'),opacity=0.8,name='expt_data') # norm_T2_X32 over sqrt T1 signal
for i in range(n_plot):
  fig.add_scatter(x=t*1e6,y=c[rand_set[i],:],line=dict(width=4,color=color_str[i]),opacity=0.8,name=f'{rand_set[i]} smooth')
  fig.add_scatter(x=T_train*1e6,y=c_train[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'{rand_set[i]} noisy')
  fig.add_scatter(x=t*1e6,y=stretchExp(t,fit_par[0][rand_set[i]]*1e-6,fit_par[1][rand_set[i]],1),line=dict(width=2,color=color_str[i]),opacity=0.8,name=f'Fit on smooth')
  # fig.add_scatter(x=T_in*1e6,y=stretchExp(T_in,fit_par[0][rand_set[i]]*1e-6,fit_par[1][rand_set[i]],1),line=dict(width=2,color=color_str[i]),opacity=0.8,name=f'Fit')
fig.update_layout(template=fig_template,  title = 'Coherence Curves',
                  width = 600,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,720]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
# n_plot = 10
# rand_set = np.random.randint(0,s_in.shape[0],(n_plot,))
fig = go.Figure()
for i in range(n_plot):
  fig.add_scatter(x=1e-6*w_in/(2*np.pi),y=s_in[rand_set[i],:],line=dict(width=2),opacity=0.8,name=f'{rand_set[i]}')
  # fig.add_scatter(x=1e-6*w_test/(2*np.pi),y=s_test[rand_set[i],:],line=dict(width=2),opacity=0.8,name=f'Test {rand_set[i]}')
  # fig.add_scatter(x=1e-6*w_test2/(2*np.pi),y=s_test2[rand_set[i],:],line=dict(width=2),opacity=0.8,name=f'Test2 {rand_set[i]}')
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=s_train[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'{rand_set[i]}')

fig.update_layout(template=fig_template,  title = 'Noise Spectra',
                  width = 600,
                  xaxis = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                               type="log",
                              #  range=[5,50],
                               ),
                  yaxis = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                               type="log",

                               ),
                 )
fig

In [ ]:
# Filter function
t_f = np.geomspace(1e-6, 100e-6, 10)  # Total evolution time
pulses_f = 32                          # Number of pulses
fig = go.Figure()
for i in range(t_f.size):
  if pulses_f == 1:
    fig.add_scatter(x=1e-6*w,y=getFilter(1,w,piLength,t_f[i]),name=f'{np.round(t_f[i]*1e6,1)}'+' \N{greek small letter mu}s')
    # fig.add_scatter(x=1e-6*w,y=getFilter(1,w,piLength,t_f[i])/(t_f[i]**2),name=f'{np.round(t_f[i]*1e6,1)}'+' \N{greek small letter mu}s')
  else:
    fig.add_scatter(x=1e-6*w,y=getFilter(pulses_f,w,piLength,t_f[i]*pulses_f),name=f'{np.round(t_f[i]*pulses_f*1e6,1)}'+' \N{greek small letter mu}s')
    # fig.add_scatter(x=1e-6*w,y=getFilter(pulses_f,w,piLength,t_f[i]*pulses_f)/(t_f[i]*pulses_f)**2,name=f'{np.round(t_f[i]*pulses_f*1e6,1)}'+' \N{greek small letter mu}s')

fig.update_layout(template=fig_template,  title = 'Filter Functions',
                  width = 800,
                  xaxis = dict(title='\N{greek small letter omega} (MHz)',range=[0,10]),
                  yaxis = dict(title='F(\N{greek small letter omega}t)/\N{greek small letter omega}<sup>2</sup>'),
                 )
fig

In [ ]:
fig = go.Figure()
for i in range(c_train.shape[0]):
    fig.add_scatter(x=T_train*1e6,y=c_train[i,:],line=dict(width=1,color='grey'),opacity=0.2)
for j in exp_list:
  fig.add_scatter(x=expt_data['xval']*1e6,y=norm_X32_data[j,:]/np.sqrt(expt_data['pop_t1'])[j,:],line=dict(width=2,color='blue'),opacity=0.8)

fig.update_layout(template=fig_template,  title = 'Coherence Curves',
                  width = 800,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,720]),
                  yaxis = dict(title='Coherence'),
                 )

# Import Noisy training data

In [19]:
noisy_train_data = np.load('/Users/bhavesh/bhavesh/codes/ML_DD-main/data/Mar6_x32_noisy.npz',allow_pickle=True)

In [20]:
noisy_train_data.files

['c_in', 'T_in', 's_in', 'w_in', 'T_train', 'w_train', 'c_train']

In [ ]:
c_in = noisy_train_data['c_in'] # 
T_in = noisy_train_data['T_in']
s_in = noisy_train_data['s_in']
w_in = noisy_train_data['w_in']
T_train = noisy_train_data['T_train']
w_train = noisy_train_data['w_train']
c_train = noisy_train_data['c_train']